# 03: Visualization

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Austfi/xsnowForPatrol/blob/main/notebooks/03_visualization.ipynb)

Visualizing snowpack data is crucial for understanding patterns and communicating results. This notebook shows you how to create effective plots with xsnow data.

## What You'll Learn

- Plotting snow profiles (depth vs properties)
- Creating time series plots
- Multi-panel figures
- Customizing plots for presentations
- Temperature and density profiles


## Installation (For Colab Users)

If you're using Google Colab, run the cell below to install xsnow and dependencies. If you're running locally and have already installed xsnow, you can skip this cell.


In [ ]:

%pip install -q numpy pandas xarray matplotlib seaborn dask netcdf4
%pip install -q git+https://gitlab.com/avacollabra/postprocessing/xsnow



In [ ]:
import xsnow
import matplotlib.pyplot as plt
import numpy as np

# Load xsnow's lightweight sample time series dataset
print("Loading xsnow sample data for visualization...")
print("Using xsnow.single_profile_timeseries()")

try:
    ds = xsnow.single_profile_timeseries()
    print("✅ Data loaded for visualization")
    print(f"Dataset dimensions: {dict(ds.dims)}")
except Exception as e:
    print(f"⚠️ Error loading sample data: {e}")
    print("Make sure xsnow is properly installed:")
    print("  pip install git+https://gitlab.com/avacollabra/postprocessing/xsnow")
    ds = None

# Set up matplotlib for better-looking plots
plt.style.use('default')  # You can use 'seaborn' or other styles
%matplotlib inline


## Part 1: Plotting a Single Snow Profile

A snow profile shows properties (like density, temperature) as a function of depth. This is the classic "snow pit" visualization.


In [ ]:
# Select a single profile (first location, first time)
profile = ds.isel(location=0, time=0, slope=0, realization=0)

# Get depth (z coordinate) - convert to positive depth for plotting
if 'z' in profile.coords:
    depth = -profile.coords['z'].values  # Make positive (depth from surface)
    
    # Plot temperature profile
    if 'temperature' in profile.data_vars:
        fig, ax = plt.subplots(figsize=(6, 8))
        temp = profile['temperature'].values
        
        # Plot temperature vs depth
        ax.plot(temp, depth, 'r-', linewidth=2, label='Temperature')
        ax.axvline(x=0, color='k', linestyle='--', alpha=0.3)  # 0°C reference
        
        ax.set_xlabel('Temperature (°C)', fontsize=12)
        ax.set_ylabel('Depth from surface (m)', fontsize=12)
        ax.set_title('Temperature Profile', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        ax.legend()
        ax.invert_yaxis()  # Surface at top
        
        plt.tight_layout()
        plt.show()


### Density Profile

Let's also plot density:


In [ ]:
profile = ds.isel(location=0, time=0, slope=0, realization=0)

if 'z' in profile.coords and 'density' in profile.data_vars:
    depth = -profile.coords['z'].values
    density = profile['density'].values
    
    fig, ax = plt.subplots(figsize=(6, 8))
    ax.plot(density, depth, 'b-', linewidth=2, label='Density')
    
    ax.set_xlabel('Density (kg/m³)', fontsize=12)
    ax.set_ylabel('Depth from surface (m)', fontsize=12)
    ax.set_title('Density Profile', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend()
    ax.invert_yaxis()
    
    plt.tight_layout()
    plt.show()


## Part 2: Multi-Panel Profile Plot

Combine multiple properties in one figure (like a real snow pit diagram):


In [ ]:
profile = ds.isel(location=0, time=0, slope=0, realization=0)

if 'z' in profile.coords:
    depth = -profile.coords['z'].values
    
    # Create figure with multiple subplots
    fig, axes = plt.subplots(1, 3, figsize=(15, 8), sharey=True)
    
    # Temperature
    if 'temperature' in profile.data_vars:
        axes[0].plot(profile['temperature'].values, depth, 'r-', linewidth=2)
        axes[0].axvline(x=0, color='k', linestyle='--', alpha=0.3)
        axes[0].set_xlabel('Temperature (°C)', fontsize=11)
        axes[0].set_title('Temperature', fontsize=12, fontweight='bold')
        axes[0].grid(True, alpha=0.3)
        axes[0].invert_yaxis()
    
    # Density
    if 'density' in profile.data_vars:
        axes[1].plot(profile['density'].values, depth, 'b-', linewidth=2)
        axes[1].set_xlabel('Density (kg/m³)', fontsize=11)
        axes[1].set_title('Density', fontsize=12, fontweight='bold')
        axes[1].grid(True, alpha=0.3)
        axes[1].invert_yaxis()
    
    # Add a third plot (e.g., grain size if available)
    if 'grain_size' in profile.data_vars:
        axes[2].plot(profile['grain_size'].values, depth, 'g-', linewidth=2)
        axes[2].set_xlabel('Grain Size (mm)', fontsize=11)
        axes[2].set_title('Grain Size', fontsize=12, fontweight='bold')
        axes[2].grid(True, alpha=0.3)
        axes[2].invert_yaxis()
else:
    print("""
    Multi-panel plot example:
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 8), sharey=True)
    
    # Plot temperature, density, grain_size side by side
    # All share the same y-axis (depth)
    """)


## Part 3: Time Series Plots

Plot how snowpack properties change over time:


In [ ]:
# Get time series of snow height
hs_series = ds['HS'].isel(location=0, slope=0, realization=0)
times = ds.coords['time'].values

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(times, hs_series.values, 'b-', linewidth=2, label='Snow Height (HS)')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Snow Height (m)', fontsize=12)
ax.set_title('Snow Height Time Series', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()

# Rotate x-axis labels for readability
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### Surface Temperature Time Series

Track surface layer temperature over time:


In [ ]:
# Get surface temperature (layer 0) over time
surface_temp = ds['temperature'].isel(location=0, layer=0, slope=0, realization=0)
times = ds.coords['time'].values

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(times, surface_temp.values, 'r-', linewidth=2, label='Surface Temperature')
ax.axhline(y=0, color='k', linestyle='--', alpha=0.3, label='Freezing Point')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Temperature (°C)', fontsize=12)
ax.set_title('Surface Temperature Time Series', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend()

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## Part 4: Heatmap (Depth-Time)

A heatmap shows how properties change with both depth and time - very useful for seeing evolution:


In [ ]:
# Select one location
temp_data = ds['temperature'].isel(location=0, slope=0, realization=0)

# Get coordinates
times = ds.coords['time'].values
z_coords = ds.coords['z'].isel(location=0, time=0, slope=0, realization=0).values
depth = -z_coords  # Convert to positive depth

# Create heatmap
fig, ax = plt.subplots(figsize=(14, 8))

# Plot as filled contour or pcolormesh
im = ax.contourf(times, depth, temp_data.values.T, levels=20, cmap='RdYlBu_r')

# Add colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Temperature (°C)', fontsize=12)

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Depth from surface (m)', fontsize=12)
ax.set_title('Temperature Evolution (Depth-Time)', fontsize=14, fontweight='bold')
ax.invert_yaxis()  # Surface at top

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## Part 5: Comparing Multiple Locations

If you have data from multiple locations, compare them:


In [ ]:
# Check if we have multiple locations
n_locations = ds.dims.get('location', 0)

if n_locations > 1:
    times = ds.coords['time'].values
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot each location
    for i in range(n_locations):
        location_name = ds.coords['location'].values[i]
        hs_series = ds['HS'].isel(location=i, slope=0, realization=0)
        ax.plot(times, hs_series.values, linewidth=2, label=f'Location {location_name}')
    
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Snow Height (m)', fontsize=12)
    ax.set_title('Snow Height Comparison', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("""
    Comparing locations:
    
    # Plot each location
    for loc in ds.coords['location'].values:
        hs = ds['HS'].sel(location=loc)
        plt.plot(ds.coords['time'], hs.values, label=loc)
    
    plt.legend()
    plt.show()
    """)


## Part 6: Customizing Plots

Make publication-quality figures:


In [ ]:
# Example of a highly customized plot
profile = ds.isel(location=0, time=0, slope=0, realization=0)
depth = -profile.coords['z'].values
temp = profile['temperature'].values

# Create figure with custom styling
fig, ax = plt.subplots(figsize=(8, 10))

# Plot with custom style
ax.plot(temp, depth, 'r-', linewidth=3, marker='o', markersize=6, 
        label='Temperature', alpha=0.8)
ax.axvline(x=0, color='k', linestyle='--', linewidth=1.5, alpha=0.5, label='Freezing')

# Customize axes
ax.set_xlabel('Temperature (°C)', fontsize=14, fontweight='bold')
ax.set_ylabel('Depth from surface (m)', fontsize=14, fontweight='bold')
ax.set_title('Snow Temperature Profile', fontsize=16, fontweight='bold', pad=20)

# Customize grid
ax.grid(True, alpha=0.3, linestyle=':', linewidth=0.5)
ax.set_axisbelow(True)

# Customize ticks
ax.tick_params(labelsize=11)

# Legend
ax.legend(fontsize=12, framealpha=0.9)

# Invert y-axis
ax.invert_yaxis()

# Add text annotation
ax.text(0.05, 0.95, 'Surface', transform=ax.transAxes, 
        fontsize=10, verticalalignment='top', 
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()


## Summary

✅ **What we learned:**

1. **Single profiles**: Plot temperature/density vs depth
2. **Multi-panel plots**: Combine multiple properties
3. **Time series**: Track changes over time
4. **Heatmaps**: Visualize depth-time evolution
5. **Comparisons**: Compare multiple locations
6. **Customization**: Make publication-quality figures

## Key Plotting Tips

- Always invert y-axis for depth plots (surface at top)
- Use `sharey=True` for multi-panel profile plots
- Rotate date labels for readability: `plt.xticks(rotation=45)`
- Save high-resolution figures: `plt.savefig('plot.png', dpi=300)`
- Use appropriate colormaps for heatmaps (e.g., 'RdYlBu_r' for temperature)

## Next Steps

Ready for advanced analysis? Move on to:
- **04_advanced_analysis.ipynb**: Stability indices, hazard calculations, and more

## Exercises

1. Create a temperature profile for a specific date
2. Plot snow height over the entire time period
3. Create a heatmap showing density evolution
4. Compare surface temperature between two different dates
5. Make a publication-quality multi-panel figure with 4 subplots
